# Setup

In [1]:
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

In [2]:
# Standard library imports
import os
import re
import math
import json
import random
import functools
from datetime import datetime, timedelta
from typing import Any, Callable, Dict, List, Optional, Tuple
from collections import OrderedDict

# Third-party imports
import torch
import openai
import faiss
import tenacity

# LangChain imports
from langchain.utils import mock_now
from langchain.docstore import InMemoryDocstore
from langchain.retrievers import TimeWeightedVectorStoreRetriever
from langchain_community.vectorstores import FAISS
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.chains import LLMChain
from langchain_core.language_models import BaseLanguageModel
from langchain.prompts import PromptTemplate
from langchain.schema import HumanMessage, SystemMessage, BaseMemory, Document
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.output_parsers import RegexParser

# Pydantic imports
from pydantic import BaseModel, Field, ConfigDict

# Hugging Face imports
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoConfig, pipeline, AutoModel
from peft import PeftModel, PeftConfig
from langchain_huggingface import HuggingFacePipeline

In [3]:
# Set API Keys
from kaggle_secrets import UserSecretsClient # API Loggins
user_secrets = UserSecretsClient()

## Hugging Face
Hugging_Face_token = user_secrets.get_secret("Hugging_Face_token")

## Openai
OPENAI_API_KEY = user_secrets.get_secret("OPENAI_API_KEY")

In [4]:
# Login to Hugging Face
from huggingface_hub import login

login(Hugging_Face_token)

# Load Model

## GPT 3.5

In [5]:
LLM_gpt = ChatOpenAI(model="gpt-3.5-turbo", 
                 max_tokens=1500, 
                 api_key = OPENAI_API_KEY) 

In [6]:
selected_embeddings_model = OpenAIEmbeddings(api_key = OPENAI_API_KEY)
embedding_size_selectedLLM = len(selected_embeddings_model.embed_query("This is a test."))
print(f"Embedding size: {embedding_size_selectedLLM}")

Embedding size: 1536


## Llama3.2 - Finetuned

In [7]:
# Define model paths
BASE_MODEL_ID = "meta-llama/Llama-3.2-3B-Instruct"

FINETUNED_MODEL_PATH_ChristineJardine = "/kaggle/input/finetune-llama-v3-christinejardine/kaggle/working/fine-tuned-llama_ChristineJardine"
FINETUNED_MODEL_PATH_TimFarron = "/kaggle/input/finetune-llama-v3-timfarron/kaggle/working/fine-tuned-llama_TimFarron"
FINETUNED_MODEL_PATH_JoStevens = "/kaggle/input/finetune-llama-v3-jostevens/kaggle/working/fine-tuned-llama_JoStevens"
FINETUNED_MODEL_PATH_SteveDouble = "/kaggle/input/finetune-llama-v3-stevedouble/kaggle/working/fine-tuned-llama_SteveDouble"
FINETUNED_MODEL_PATH_RachaelMaskell = "/kaggle/input/finetune-llama-v3-rachaelmaskell/kaggle/working/fine-tuned-llama_RachaelMaskell"
FINETUNED_MODEL_PATH_KitMalthouse = "/kaggle/input/finetune-llama-v3-kitmalthouse/kaggle/working/fine-tuned-llama_KitMalthouse"

In [8]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

In [9]:
class DebateLLM:
    def __init__(self, base_model_id, lora_adapter_path, bnb_config):
        # Load the tokenizer from the base model
        self.tokenizer = AutoTokenizer.from_pretrained(base_model_id, use_fast=True)
        self.tokenizer.pad_token = self.tokenizer.eos_token
        
        # Load the base model with quantization configuration
        base_model = AutoModelForCausalLM.from_pretrained(
            base_model_id,
            quantization_config=bnb_config,
            device_map="auto"
        )
        
        # Load the LoRA adapter weights onto the base model
        self.model = PeftModel.from_pretrained(base_model, lora_adapter_path)
        self.model.eval()

debate_LLM_ChristineJardine = DebateLLM(BASE_MODEL_ID, FINETUNED_MODEL_PATH_ChristineJardine, bnb_config)
debate_LLM_TimFarron        = DebateLLM(BASE_MODEL_ID, FINETUNED_MODEL_PATH_TimFarron, bnb_config)
debate_LLM_JoStevens        = DebateLLM(BASE_MODEL_ID, FINETUNED_MODEL_PATH_JoStevens, bnb_config)
debate_LLM_SteveDouble      = DebateLLM(BASE_MODEL_ID, FINETUNED_MODEL_PATH_SteveDouble, bnb_config)
debate_LLM_RachaelMaskell   = DebateLLM(BASE_MODEL_ID, FINETUNED_MODEL_PATH_RachaelMaskell, bnb_config)
debate_LLM_KitMalthouse     = DebateLLM(BASE_MODEL_ID, FINETUNED_MODEL_PATH_KitMalthouse, bnb_config)

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

# Generative AI Setup
The [codes](https://python.langchain.com/api_reference/experimental/generative_agents.html) for the classes `GenerativeAgentMemory` and `GenerativeAgent` was entirely reused from the **[LangChain Experimental](https://pypi.org/project/langchain-experimental/)** project in the LangChain Python API reference - intended for research and experimental uses, with a few minor tweaks and proper configuration of the prompts.


## Generative Agent Memory

In [10]:
class GenerativeAgentMemory(BaseMemory):
    """Memory for the generative agent."""
    
    llm: BaseLanguageModel
    """The core language model."""
    
    memory_retriever: TimeWeightedVectorStoreRetriever
    """The retriever to fetch related memories."""
    
    verbose: bool = False
    reflection_threshold: Optional[float] = None
    """When aggregate_importance exceeds reflection_threshold, stop to reflect."""
    
    current_plan: List[str] = []
    """The current plan of the agent."""
    
    # A weight of 0.15 makes this less important than it
    # would be otherwise, relative to salience and time
    importance_weight: float = 0.15
    
    """How much weight to assign the memory importance."""
    aggregate_importance: float = 0.0  # : :meta private:
    
    """Track the sum of the 'importance' of recent memories.
    Triggers reflection when it reaches reflection_threshold."""
    max_tokens_limit: int = 2000  # : :meta private:
    
    # input keys
    queries_key: str = "queries"
    most_recent_memories_token_key: str = "recent_memories_token"
    add_memory_key: str = "add_memory"
    
    # output keys
    relevant_memories_key: str = "relevant_memories"
    relevant_memories_simple_key: str = "relevant_memories_simple"
    most_recent_memories_key: str = "most_recent_memories"
    now_key: str = "now"
    reflecting: bool = False
    
    def chain(self, prompt: PromptTemplate) -> LLMChain:
        return LLMChain(llm=self.llm, prompt=prompt, verbose=self.verbose)
    @staticmethod
    
    def _parse_list(text: str) -> List[str]:
        """Parse a newline-separated string into a list of strings."""
        lines = re.split(r"\n", text.strip())
        lines = [line for line in lines if line.strip()]  # remove empty lines
        return [re.sub(r"^\s*\d+\.\s*", "", line).strip() for line in lines]
    
    def _get_topics_of_reflection(self, last_k: int = 50) -> List[str]:
        """Return the 3 most salient high-level questions about recent observations."""
        prompt = PromptTemplate.from_template(
            "{observations}\n\n"
            "Given only the information above, what are the 3 most salient "
            "high-level questions we can answer about the subjects in the statements?\n"
            "Provide each question on a new line."
        )
        observations = self.memory_retriever.memory_stream[-last_k:]
        observation_str = "\n".join(
            [self._format_memory_detail(o) for o in observations]
        )
        result = self.chain(prompt).run(observations=observation_str)
        return self._parse_list(result)
    
    def _get_insights_on_topic(
        self, topic: str, now: Optional[datetime] = None
    ) -> List[str]:
        """Generate 'insights' on a topic of reflection, based on pertinent memories."""
        prompt = PromptTemplate.from_template(
            "Statements relevant to: '{topic}'\n"
            "---\n"
            "{related_statements}\n"
            "---\n"
            "What 5 high-level novel insights can you infer from the above statements "
            "that are relevant for answering the following question?\n"
            "Do not include any insights that are not relevant to the question.\n"
            "Do not repeat any insights that have already been made.\n\n"
            "Question: {topic}\n\n"
            "(example format: insight (because of 1, 5, 3))\n"
        )
        related_memories = self.fetch_memories(topic, now=now)
        related_statements = "\n".join(
            [
                self._format_memory_detail(memory, prefix=f"{i+1}. ")
                for i, memory in enumerate(related_memories)
            ]
        )
        result = self.chain(prompt).run(
            topic=topic, related_statements=related_statements
        )
        # TODO: Parse the connections between memories and insights
        return self._parse_list(result)
    
    def pause_to_reflect(self, now: Optional[datetime] = None) -> List[str]:
        """Reflect on recent observations and generate 'insights'."""
        if self.verbose:
            logger.info("Character is reflecting")
        new_insights = []
        topics = self._get_topics_of_reflection()
        for topic in topics:
            insights = self._get_insights_on_topic(topic, now=now)
            for insight in insights:
                self.add_memory(insight, now=now)
            new_insights.extend(insights)
        return new_insights
    
    def _score_memory_importance(self, memory_content: str) -> float:
        """Score the absolute importance of the given memory."""
        prompt = PromptTemplate.from_template(
            "On the scale of 1 to 10, where 1 is purely mundane"
            + " (e.g., brushing teeth, making bed) and 10 is"
            + " extremely poignant (e.g., a break up, college"
            + " acceptance), rate the likely poignancy of the"
            + " following piece of memory. Respond with a single integer."
            + "\nMemory: {memory_content}"
            + "\nRating: "
        )
        score = self.chain(prompt).run(memory_content=memory_content).strip()
        if self.verbose:
            logger.info(f"Importance score: {score}")
        match = re.search(r"^\D*(\d+)", score)
        if match:
            return (float(match.group(1)) / 10) * self.importance_weight
        else:
            return 0.0
    
    def _score_memories_importance(self, memory_content: str) -> List[float]:
        """Score the absolute importance of the given memory."""
        prompt = PromptTemplate.from_template(
            "On the scale of 1 to 10, where 1 is purely mundane"
            + " (e.g., brushing teeth, making bed) and 10 is"
            + " extremely poignant (e.g., a break up, college"
            + " acceptance), rate the likely poignancy of the"
            + " following piece of memory. Always answer with only a list of numbers."
            + " If just given one memory still respond in a list."
            + " Memories are separated by semi colans (;)"
            + "\nMemories: {memory_content}"
            + "\nRating: "
        )
        scores = self.chain(prompt).run(memory_content=memory_content).strip()
        if self.verbose:
            logger.info(f"Importance scores: {scores}")
        # Split into list of strings and convert to floats
        scores_list = [float(x) for x in scores.split(";")]
        return scores_list
    
    def add_memories(
        self, memory_content: str, now: Optional[datetime] = None
    ) -> List[str]:
        """Add an observations or memories to the agent's memory."""
        importance_scores = self._score_memories_importance(memory_content)
        self.aggregate_importance += max(importance_scores)
        memory_list = memory_content.split(";")
        documents = []
        for i in range(len(memory_list)):
            documents.append(
                Document(
                    page_content=memory_list[i],
                    metadata={"importance": importance_scores[i]},
                )
            )
        result = self.memory_retriever.add_documents(documents, current_time=now)
        # After an agent has processed a certain amount of memories (as measured by
        # aggregate importance), it is time to reflect on recent events to add
        # more synthesized memories to the agent's memory stream.
        if (
            self.reflection_threshold is not None
            and self.aggregate_importance > self.reflection_threshold
            and not self.reflecting
        ):
            self.reflecting = True
            self.pause_to_reflect(now=now)
            # Hack to clear the importance from reflection
            self.aggregate_importance = 0.0
            self.reflecting = False
        return result
    
    def add_memory(
        self, memory_content: str, now: Optional[datetime] = None) -> List[str]:
        """Add an observation or memory to the agent's memory."""
        
        importance_score = self._score_memory_importance(memory_content)
        
        self.aggregate_importance += importance_score
        
        document = Document(page_content=memory_content, 
                            metadata={"importance": importance_score} )
        
        result = self.memory_retriever.add_documents([document], current_time=now)
        
        # After an agent has processed a certain amount of memories (as measured by
        # aggregate importance), it is time to reflect on recent events to add
        # more synthesized memories to the agent's memory stream.
        if (
            self.reflection_threshold is not None
            and self.aggregate_importance > self.reflection_threshold
            and not self.reflecting
        ):
            self.reflecting = True
            self.pause_to_reflect(now=now)
            # Hack to clear the importance from reflection
            self.aggregate_importance = 0.0
            self.reflecting = False
            
        return result
    
    def fetch_memories(
        self, observation: str, now: Optional[datetime] = None
    ) -> List[Document]:
        """Fetch related memories."""
        if now is not None:
            with mock_now(now):
                return self.memory_retriever.invoke(observation)
        else:
            return self.memory_retriever.invoke(observation)
    
    def format_memories_detail(self, relevant_memories: List[Document]) -> str:
        content = []
        for mem in relevant_memories:
            content.append(self._format_memory_detail(mem, prefix="- "))
        return "\n".join([f"{mem}" for mem in content])
    
    def _format_memory_detail(self, memory: Document, prefix: str = "") -> str:
        created_time = memory.metadata["created_at"].strftime("%B %d, %Y, %I:%M %p")
        return f"{prefix}[{created_time}] {memory.page_content.strip()}"
    
    def format_memories_simple(self, relevant_memories: List[Document]) -> str:
        return "; ".join([f"{mem.page_content}" for mem in relevant_memories])
    
    def _get_memories_until_limit(self, consumed_tokens: int) -> str:
        """Reduce the number of tokens in the documents."""
        result = []
        for doc in self.memory_retriever.memory_stream[::-1]:
            if consumed_tokens >= self.max_tokens_limit:
                break
            consumed_tokens += self.llm.get_num_tokens(doc.page_content)
            if consumed_tokens < self.max_tokens_limit:
                result.append(doc)
        return self.format_memories_simple(result)
    
    def memory_variables(self) -> List[str]:
        """Input keys this memory class will load dynamically."""
        return []
   
    def load_memory_variables(self, inputs: Dict[str, Any]) -> Dict[str, str]:
        """Return key-value pairs given the text input to the chain."""
        queries = inputs.get(self.queries_key)
        now = inputs.get(self.now_key)
        if queries is not None:
            relevant_memories = [
                mem for query in queries for mem in self.fetch_memories(query, now=now)
            ]
            return {
                self.relevant_memories_key: self.format_memories_detail(
                    relevant_memories
                ),
                self.relevant_memories_simple_key: self.format_memories_simple(
                    relevant_memories
                ),
            }
        most_recent_memories_token = inputs.get(self.most_recent_memories_token_key)
        if most_recent_memories_token is not None:
            return {
                self.most_recent_memories_key: self._get_memories_until_limit(
                    most_recent_memories_token
                )
            }
        return {}
    
    def save_context(self, inputs: Dict[str, Any], outputs: Dict[str, Any]) -> None:
        """Save the context of this model run to memory."""
        # TODO: fix the save memory key
        mem = outputs.get(self.add_memory_key)
        now = outputs.get(self.now_key)
        if mem:
            self.add_memory(mem, now=now)
    
    def clear(self) -> None:
        """Clear memory contents."""
        # TODO

## Generative Agent

In [11]:
class GenerativeAgent(BaseModel):
    """Agent as a character with memory and innate characteristics."""
    name: str
    """The character's name."""
    
    age: Optional[int] = None
    """The optional age of the character."""
    
    traits: str = "N/A"
    """Permanent traits to ascribe to the character."""
    
    status: str
    """The traits of the character you wish not to change."""
    
    memory: GenerativeAgentMemory
    """The memory object that combines relevance, recency, and 'importance'."""
    
    llm: BaseLanguageModel
    llm_debate: Any
    """The underlying language model."""

    
    verbose: bool = False
    summary: str = ""  #: :meta private:
    """Stateful self-summary generated via reflection on the character's memory."""
    
    summary_refresh_seconds: int = 3600  #: :meta private:
    """How frequently to re-generate the summary."""
    
    last_refreshed: datetime = Field(default_factory=datetime.now)  # : :meta private:
    """The last time the character's summary was regenerated."""
    
    daily_summaries: List[str] = Field(default_factory=list)  # : :meta private:
    """Summary of the events in the plan that the agent took."""
    
    model_config = ConfigDict(
        arbitrary_types_allowed=True,
    )
    
## LLM-related methods
    def _parse_list(text: str) -> List[str]:
        """Parse a newline-separated string into a list of strings."""
        lines = re.split(r"\n", text.strip())
        return [re.sub(r"^\s*\d+\.\s*", "", line).strip() for line in lines]
        
    def chain(self, 
              prompt: PromptTemplate, 
              llm: Optional[BaseLanguageModel] = None) -> LLMChain:
        """Create a chain with the same settings as the agent."""
        
        return LLMChain(
            llm= llm or self.llm,       # If llm is None, it will use self.llm
            prompt=prompt, 
            verbose=self.verbose, 
            memory=self.memory
        )
        
    def _get_entity_from_observation(self, observation: str) -> str:
        prompt = PromptTemplate.from_template(
            "What is the observed entity in the following observation? {observation}"
            + "\nEntity="
        )
        return self.chain(prompt).run(observation=observation).strip()
        
    def _get_entity_action(self, observation: str, entity_name: str) -> str:
        prompt = PromptTemplate.from_template(
            "What is the {entity} doing in the following observation? {observation}"
            + "\nThe {entity} is"
        )
        return (
            self.chain(prompt).run(entity=entity_name, observation=observation).strip()
        )

## Summarize Most relevant memories
    def summarize_related_memories(self, observation: str) -> str:
        """Summarize memories that are most relevant to an observation."""
        prompt = PromptTemplate.from_template(
            """
            {q1}?
            Context from memory:
            {relevant_memories}
            Relevant context: 
            """
        )
        entity_name = self._get_entity_from_observation(observation)
        entity_action = self._get_entity_action(observation, entity_name)
        q1 = f"What is the relationship between {self.name} and {entity_name}"
        q2 = f"{entity_name} is {entity_action}"
        return self.chain(prompt=prompt).run(q1=q1, queries=[q1, q2]).strip()
        
## Generate Summary of the agent + reaction 
    def _generate_reaction(
        self, 
        observation: str, 
        suffix: str, 
        now: Optional[datetime] = None,
        llm: Optional[Any] = None,  # Expecting an object with 'model' and 'tokenizer'
    ) -> str:

        # Gather context information
        agent_summary_description = self.get_summary(now=now)
        relevant_memories_str = self.summarize_related_memories(observation)
        current_time_str = (
            datetime.now().strftime("%B %d, %Y, %I:%M %p")
            if now is None
            else now.strftime("%B %d, %Y, %I:%M %p")
        )
        
        # Construct the system message (context)
        system_content = (
            f"{agent_summary_description}\n"
            f"It is {current_time_str}.\n"
            f"{self.name}'s status: {self.status}\n"
            f"Summary of relevant context from {self.name}'s memory:\n{relevant_memories_str}\n"
            f"\n\n"
            f"{suffix}"
        )

        # Build messages in chat format
        messages = [
            {"role": "system", "content": system_content},
            {"role": "user", "content": observation},
        ]
        
        # Build the prompt using the tokenizer's chat template function.
        prompt = llm.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
        
        # Tokenize the prompt manually
        inputs = llm.tokenizer(prompt, return_tensors="pt", 
                               padding=True, truncation=True).to("cuda")
        
        # Generate output using model.generate; adjust generation parameters as needed.
        outputs = llm.model.generate(**inputs, max_new_tokens=300, num_return_sequences=1)
        
        # Decode the generated text
        generated_text = llm.tokenizer.decode(outputs[0], skip_special_tokens=True)
        generated_text = generated_text.replace('\n', ' ')

        # Clean the output response
        match = re.search(r"assistant\s*(.*)", generated_text, re.IGNORECASE | re.DOTALL)
        if match:
            return match.group(1).strip()
        else:
            return generated_text.strip()
    
## Generate Dialogue response
    def generate_dialogue_response(self, observation: str, now: Optional[datetime] = None) -> str:

        call_to_action_template = (
            f"You are {self.name}, a Member of Parliament responding in a debate session in the UK House of Commons.\n"
            f"Act as {self.name} would, using your distinct voice and perspective.\n"
            f"Respond to the input.\n"
            f"Ensure that your response is direct, fully in character, and reflects your established views and tone.\n"
            f"Respond exactly as {self.name} would speak in this context."
        )
        
        response_text = self._generate_reaction(observation = observation,
                                                suffix = call_to_action_template,
                                                now=now, llm=self.llm_debate)
        
        # Save the dialogue turn to memory
        self.memory.save_context(
            {},
            {
                self.memory.add_memory_key: f"{self.name} observed {observation} and said {response_text}",
                self.memory.now_key: now,
            },
        )
        return f"{self.name} said {response_text}"

## Decide if the agent wants to respond to the observation
    def decide_to_respond(self, observation: str, 
                          now: Optional[datetime] = None,
                          threshold: float = 7.0) -> bool:

        call_to_action_template = (
            f"You are {self.name}, a Member of Parliament (MP) currently sitting in the UK House of Commons."
            f"On a scale of 1 to 10, how salient is the following statement to you as an MP?"
            f"\n- 1: Not relevant at all"
            f"\n- 10: Extremely relevant"
            f"\n\nRespond ONLY with a single integer between 1 and 10!"
            )
        
        full_result = self._generate_reaction(observation = observation,
                                              suffix = call_to_action_template,
                                              now=now, llm = self.llm_debate)

        match = re.search(r'\d+', full_result) # Searches for the first occurrence of a 'digit'
        if match:
            relevance_score = int(match.group())
        else:
            print(f"Unexpected non-numeric response from agent: {full_result}") 
            relevance_score = 0

        # Save the decision context to memory
        self.memory.save_context(
            {},
            {
                self.memory.add_memory_key: f"In assessing the observation: '{observation}', "
                f"{self.name} rated its relevance as {relevance_score} on a scale of 1 to 10, where 1 is 'not relevant at all' and 10 is 'extremely relevant'",
                self.memory.now_key: now,
            },
        )
         
        # Return True if the relevance score is equal or above the threshold, else False
        return relevance_score >= threshold
    
    ######################################################
    # Agent stateful' summary methods.                   #
    # Each dialog or response prompt includes a header   #
    # summarizing the agent's self-description. This is  #
    # updated periodically through probing its memories  #
    ######################################################
    
    def _compute_agent_summary(self) -> str:
        """"""
        prompt = PromptTemplate.from_template(
            "How would you summarize {name}'s core characteristics given the"
            + " following statements:\n"
            + "{relevant_memories}"
            + "Do not embellish."
            + "\n\nSummary: "
        )
        # The agent seeks to think about their core characteristics.
        return (
            self.chain(prompt)
            .run(name=self.name, queries=[f"{self.name}'s core characteristics"])
            .strip()
        )

## Get Summary of the agent
    def get_summary(
        self, force_refresh: bool = False, now: Optional[datetime] = None
    ) -> str:
        """Return a descriptive summary of the agent."""
        current_time = datetime.now() if now is None else now
        since_refresh = (current_time - self.last_refreshed).seconds
        if (
            not self.summary
            or since_refresh >= self.summary_refresh_seconds
            or force_refresh
        ):
            self.summary = self._compute_agent_summary()
            self.last_refreshed = current_time
        age = self.age if self.age is not None else "N/A"
        return (
            f"Name: {self.name} (age: {age})"
            + f"\nInnate traits: {self.traits}"
            + f"\n{self.summary}"
        )
    
    def get_full_header(
        self, force_refresh: bool = False, now: Optional[datetime] = None
    ) -> str:
        """Return a full header of the agent's status, summary, and current time."""
        now = datetime.now() if now is None else now
        summary = self.get_summary(force_refresh=force_refresh, now=now)
        current_time_str = now.strftime("%B %d, %Y, %I:%M %p")
        return (
            f"{summary}\nIt is {current_time_str}.\n{self.name}'s status: {self.status}"
        )

# Create Agent
- [GenerativeAgentMemory](https://python.langchain.com/api_reference/experimental/generative_agents/langchain_experimental.generative_agents.memory.GenerativeAgentMemory.html): **Memory** for the generative agent 
   - `llm`
   - `memory_retriever` = create_new_memory_retriever()
   - `current_plan`
   - `reflection_threshold`
   - `add_memory` add observation/memory
- [GenerativeAgent](https://python.langchain.com/api_reference/experimental/generative_agents.html): Agent as a character with **memory** and innate **characteristics**,  
   - basics like `name`, `age` and `llm`
   - `memory` object that combines relevance, recency, and ‘importance’
   - `summary` and `summary_refresh_seconds` to set how frequently to re-generate the summary
   - `summarize_related_memories`: Summarize memories that are most relevant to an observation
   - `status` fix-objectives / traits of the character you wish not to change
   - `traits` set Permanent traits to ascribe to the character 
   - `generate_dialogue_response`

In [12]:
# Relevance Score function - relevance_score_fn()
def relevance_score_fn(score: float) -> float:
    """Return a similarity score on a scale [0, 1]."""
    return 1.0 - score / math.sqrt(2)

In [13]:
# Memory Retriever function - create_new_memory_retriever()
def create_new_memory_retriever():
    """Create a new vector store retriever unique to the agent."""
    
    embeddings_model = selected_embeddings_model  
    
    # Initialize the vectorstore as empty
    embedding_size = embedding_size_selectedLLM           #use: 1536 (GPT3.5) or 3072 (Llamma)
    
    index = faiss.IndexFlatL2(embedding_size)
    vectorstore = FAISS(
        embeddings_model.embed_query,  
        index,
        InMemoryDocstore({}),  # empty Memory docstore
        {},  # index-to-document store ID mapping
        relevance_score_fn=relevance_score_fn,
    )
    
    # Time-weighted scoring mechanism
    return TimeWeightedVectorStoreRetriever(
        vectorstore=vectorstore,
        other_score_keys=["importance"],
        k=5                                   # retrieve up to ___ relevant memories
    )

In [14]:
# Agent Creation function - create_debate_agent()
def create_debate_agent(name, age, traits, status, 
                        #reflection_threshold, 
                        llm, llm_debate):
   
    memory = GenerativeAgentMemory(
        llm=llm,
        memory_retriever=create_new_memory_retriever(),
        verbose=False,
        #reflection_threshold=reflection_threshold,  # adjust as needed for reflection frequency
    )
    
    agent = GenerativeAgent(
        name=name,
        age=age,
        traits=traits,
        status=status,
        memory_retriever=create_new_memory_retriever(),
        llm=llm,
        llm_debate = llm_debate,
        memory=memory,
    )
    return agent

## Define Agent Traits

In [15]:
# Create agents for each MP
ChristineJardine = create_debate_agent(
    name="Christine Jardine",
    age=2019-1960,
    traits="Advocate for EU citizens' rights, empathetic communicator",
    status="Scottish Liberal Democrat politician; Opposes ending free movement; highlights contributions of EU nationals to the UK.",
    llm=LLM_gpt, llm_debate = debate_LLM_ChristineJardine
)

KitMalthouse = create_debate_agent(
    name="Kit Malthouse",
    age=2019-1966,
    traits="Firm on immigration control, prioritizes national sovereignty",
    status="British Conservative Party politician; Supports ending free movement; focuses on controlled immigration policies.",
    llm=LLM_gpt, llm_debate = debate_LLM_KitMalthouse
)

SteveDouble = create_debate_agent(
    name="Steve Double",
    age=2019-1966,  # Example age
    traits="Emphasizes national interests, supportive of government policies",
    status="British Conservative Party politician; Advocates for ending free movement; reassures EU citizens are welcome under new schemes.",
    llm=LLM_gpt, llm_debate = debate_LLM_SteveDouble
)

TimFarron = create_debate_agent(
    name="Tim Farron",
    age=2019-1970,  # Example age
    traits="Pro-European, champions individual rights",
    status="British Liberal Democrats Party politician; Opposes ending free movement; stresses benefits of EU integration.",
    llm=LLM_gpt, llm_debate = debate_LLM_TimFarron
)

JoStevens = create_debate_agent(
    name="Jo Stevens",
    age=2019-1966,  # Example age
    traits="Defender of workers' rights, critical of government policies",
    status="British Labour Party politician; Questions impact of ending free movement on labor markets and rights.",
    llm=LLM_gpt, llm_debate = debate_LLM_JoStevens
)

RachaelMaskell = create_debate_agent(
    name="Rachael Maskell",
    age=2019-1972,  # Example age
    traits="Advocate for social justice, focuses on community welfare",
    status="British Labour and Co-operative Party politician; Concerns about social implications of ending free movement; emphasizes inclusive policies.",
    llm=LLM_gpt, llm_debate = debate_LLM_RachaelMaskell
)

## Define Base Memories

In [16]:
# Creat Memory objects for each agent
ChristineJardine_memory = ChristineJardine.memory
KitMalthouse_memory = KitMalthouse.memory
SteveDouble_memory = SteveDouble.memory   
TimFarron_memory = TimFarron.memory
JoStevens_memory = JoStevens.memory
RachaelMaskell_memory = RachaelMaskell.memory

In [17]:
# Base Observations 
ChristineJardine_observations = [
    "Christine Jardine is a Liberal Democrat MP representing Edinburgh West.",
    "She has expressed concerns about ending free movement, highlighting its potential negative impact on public services and the economy.",
    "Jardine emphasizes the human cost of ending free movement, noting that many EU nationals in her constituency feel unwelcome and uncertain about their future."
]

# Base Observations for Kit Malthouse
KitMalthouse_observations = [
    "Kit Malthouse is a Conservative MP who has served as the Minister for Crime, Policing and the Fire Service.",
    "During debates, he has stated that the government is committed to ending free movement as part of the Brexit process.",
    "Malthouse has reassured that EU citizens residing in the UK are welcome to stay and that the government has implemented the EU Settlement Scheme to facilitate their continued residence."
]

SteveDouble_observations = [
    "Steve Double is a Conservative MP representing St Austell and Newquay.",
    "He has expressed support for ending free movement, aligning with the government's stance on controlling immigration post-Brexit.",
    "Double has emphasized that the government has provided a clear message that EU citizens currently residing in the UK are welcome to stay, citing the success of the EU Settlement Scheme."
]

TimFarron_observations = [
    "Tim Farron is a Liberal Democrat MP representing Westmorland and Lonsdale.",
    "He has criticized the decision to end free movement, arguing that it sends a negative message to EU citizens and undermines the UK's international standing.",
    "Farron has highlighted the contributions of EU nationals to the UK and has advocated for their rights to be protected post-Brexit."
]

JoStevens_observations = [
    "Jo Stevens is a Labour MP representing Cardiff Central.",
    "She has raised concerns about the impact of ending free movement on universities, noting that it could harm the UK's global reputation and deter international students and academics.",
    "Stevens has advocated for the protection of EU nationals' rights and has questioned the government's approach to immigration post-Brexit."
]

RachaelMaskell_observations = [
    "Rachael Maskell is a Labour/Co-operative MP representing York Central.",
    "She has expressed concerns about the government's immigration policies post-Brexit, particularly regarding family reunification and the rights of unaccompanied minors.",
    "Maskell has questioned the government's preparedness for the consequences of ending free movement and has called for more compassionate immigration policies."
]

In [18]:
# Loop through the observations and add to memory
tuples = [(ChristineJardine_observations, ChristineJardine.memory), 
          (KitMalthouse_observations, KitMalthouse.memory), 
          (SteveDouble_observations, SteveDouble.memory), 
          (TimFarron_observations, TimFarron.memory), 
          (JoStevens_observations, JoStevens.memory), 
          (RachaelMaskell_observations, RachaelMaskell.memory)]

for observations, memory in tuples:
    for observation in observations:
        memory.add_memory(observation)

<ipython-input-10-760dd083f2a4>:41: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  return LLMChain(llm=self.llm, prompt=prompt, verbose=self.verbose)
<ipython-input-10-760dd083f2a4>:118: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  score = self.chain(prompt).run(memory_content=memory_content).strip()


In [19]:
# View stored memories for each MP
print("Christine Jardine's stored memories:")
print(ChristineJardine_memory.memory_retriever.memory_stream)

print("\nKit Malthouse's stored memories:")
print(KitMalthouse_memory.memory_retriever.memory_stream)

print("\nSteve Double's stored memories:")
print(SteveDouble_memory.memory_retriever.memory_stream)

print("\nTim Farron's stored memories:")
print(TimFarron_memory.memory_retriever.memory_stream)

print("\nJo Stevens's stored memories:")
print(JoStevens_memory.memory_retriever.memory_stream)

print("\nRachael Maskell's stored memories:")
print(RachaelMaskell_memory.memory_retriever.memory_stream)

Christine Jardine's stored memories:
[Document(metadata={'importance': 0.015, 'last_accessed_at': datetime.datetime(2025, 3, 4, 3, 34, 14, 385507), 'created_at': datetime.datetime(2025, 3, 4, 3, 34, 14, 385507), 'buffer_idx': 0}, page_content='Christine Jardine is a Liberal Democrat MP representing Edinburgh West.'), Document(metadata={'importance': 0.045, 'last_accessed_at': datetime.datetime(2025, 3, 4, 3, 34, 15, 116418), 'created_at': datetime.datetime(2025, 3, 4, 3, 34, 15, 116418), 'buffer_idx': 1}, page_content='She has expressed concerns about ending free movement, highlighting its potential negative impact on public services and the economy.'), Document(metadata={'importance': 0.12, 'last_accessed_at': datetime.datetime(2025, 3, 4, 3, 34, 15, 723257), 'created_at': datetime.datetime(2025, 3, 4, 3, 34, 15, 723257), 'buffer_idx': 2}, page_content='Jardine emphasizes the human cost of ending free movement, noting that many EU nationals in her constituency feel unwelcome and uncer

# Create Simulation

## Input Memory

In [20]:
import pandas as pd
df_HoC_miniDebate = pd.read_csv('/kaggle/input/parlspeech/df_HoC_miniDebate.csv')
df_HoC_miniDebate.head(3)

,date,agenda,speechnumber,speaker,party,text
0,2019-10-02,Free Movement of EU Nationals,426,Christine Jardine,LibDem,"I beg to move, That this House has considered ..."
1,2019-10-02,Free Movement of EU Nationals,427,Kit Malthouse,Con,That is not correct.
2,2019-10-02,Free Movement of EU Nationals,428,Steve Double,Con,I must take exception to the language used by ...


In [21]:
initial_observation = 'Christine Jardine said, "' + df_HoC_miniDebate.iloc[0]['text'] + '"'
initial_observation

'Christine Jardine said, "I beg to move, That this House has considered proposed changes to free movement of EU nationals. I am delighted to raise the issue of freedom of movement in the EU, and I thank you, Sir David, for your chairmanship. €End freedom of movement” is a Brexiteer slogan that we have all become so accustomed to that it is easy to forget what it is really saying, and what it would really mean to this country, people living here and British citizens living abroad. We all know the basic numbers: freedom of movement allows 1.3 million British citizens to live, work, study, fall in love, marry, or retire across the European Union while more than 50,000 non-UK EU citizens work in our national health service, including support staff, nurses and doctors, all of whom play a vital role in our nation\'s health. More than 80,000 EU citizens work in social care, and even more in the UK construction industry. As the Government love to tell us, unemployment is at its lowest rate for

## Run Debate
1. Each agent add new-observation into memory. 
2. Each agent does a quick reflection on this new-observation, to whether to "respond or not respond" - depending on personal saliency (a custom function within the class `GenerativeAgent`). Output `decide_to_respond` as either True or False
3. Randomly select one agent from the list of agents that decide to respond to the observation.
4. Print this selected generate_dialogue_response as the new observation.

In [22]:
# List of agents in the debate
agents = [ChristineJardine, KitMalthouse, SteveDouble, TimFarron, JoStevens, RachaelMaskell]

In [23]:
import pandas as pd
import random
from typing import List

def run_HoC_debate_framework_3(agents: List[GenerativeAgent], 
                               initial_observation: str, 
                               save_path: str) -> None:
    """Runs a conversation between agents, enforcing a fixed order before the perturbation event."""
    
    max_turns = 25  # Adjust as needed
    turns = 0
    last_speaker = None  # Track last speaker
    
    # Define the fixed speaking order before perturbation
    fixed_speaking_order = [
        "Tim Farron", "Kit Malthouse", "Tim Farron", "Rachael Maskell",
        "Jo Stevens", "Steve Double", "Kit Malthouse", "Jo Stevens",
        "Tim Farron", "Steve Double", "Tim Farron", "Steve Double"  ]
    
    # Dictionary to map agent names to objects
    agent_dict = {agent.name: agent for agent in agents}
    
    observation = initial_observation
    print(observation)

    debate_records = []
    debate_records.append({"speaker": "Christine Jardine", "text": observation})    

    # Enforce fixed order before perturbation (Turns 1–11)
    for i, speaker_name in enumerate(fixed_speaking_order):
        if speaker_name in agent_dict:
            agent = agent_dict[speaker_name]
            last_speaker = agent  # Track last speaker
            
            # Add observation to agent memory
            agent.memory.add_memory(observation)
            
            # Generate response
            observation = agent.generate_dialogue_response(observation)
            
            # Print response to console
            print(f"{speaker_name}: {observation}\n")
            
            # Append response to the debate records list
            debate_records.append({"speaker": speaker_name, "text": observation})
        
        turns += 1  # Increment turns
    
    # Step 1.5: Introduce Perturbation Event at Turn 12
    perturbation_event = """Speaker of the House said: Before I let the Right Honourable Member continue, I must inform the House of a major development: 
    \n The US President has just confirmed the decision to impose a 25% tariff on imports coming from the UK and EU. 
    I now invite Members to continue the debate in light of this development.
    """
    
    print(f"\n PERTURBATION EVENT: \n{perturbation_event}\n")
    debate_records.append({"speaker": "Speaker", "text": perturbation_event})
    observation = perturbation_event

    # Resume normal debate flow
    while turns < max_turns:
        # Step 1: Each agent adds the new observation into memory
        for agent in agents:
            agent.memory.add_memory(observation)

        # Step 2: Select an agent who decides to respond (excluding the last speaker)
        responding_agents = [agent for agent in agents if agent.decide_to_respond(observation) and agent != last_speaker]
        
        if responding_agents:
            agent = random.choice(responding_agents)
            last_speaker = agent
            
            # Generate response
            observation = agent.generate_dialogue_response(observation)
            
            # Print response to console
            print(f"{agent.name}: {observation}\n")
            
            # Append response to the debate records list
            debate_records.append({"speaker": agent.name, "text": observation})
        
        # Increment turn count
        turns += 1
    
    # Convert debate records to a DataFrame
    df_debate = pd.DataFrame(debate_records)
    
    # Save the DataFrame as a CSV file
    df_debate.to_csv(save_path, index=False, encoding="utf-8")
    
    print(f"Debate transcript saved to {save_path}")


In [24]:
debate_output_path = "/kaggle/working/debate_transcript_llama_finetuned_perturbation.csv"

run_HoC_debate_framework_3(agents, initial_observation, debate_output_path)

Christine Jardine said, "I beg to move, That this House has considered proposed changes to free movement of EU nationals. I am delighted to raise the issue of freedom of movement in the EU, and I thank you, Sir David, for your chairmanship. €End freedom of movement” is a Brexiteer slogan that we have all become so accustomed to that it is easy to forget what it is really saying, and what it would really mean to this country, people living here and British citizens living abroad. We all know the basic numbers: freedom of movement allows 1.3 million British citizens to live, work, study, fall in love, marry, or retire across the European Union while more than 50,000 non-UK EU citizens work in our national health service, including support staff, nurses and doctors, all of whom play a vital role in our nation's health. More than 80,000 EU citizens work in social care, and even more in the UK construction industry. As the Government love to tell us, unemployment is at its lowest rate for 4

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Tim Farron: Tim Farron said Mr. Farron will speak, and I will speak as a representative of a country that has been part of the European Union for nearly 50 years. We have been part of a great and wonderful partnership, and I am proud to be part of it. I am proud to be a Liberal Democrat, and I am proud to be a Member of Parliament. I will never support the destruction of our relationship with the European Union. The Government's plans to end free movement are not just about the EU nationals living in the UK; they are about the British people who have been given the opportunity to live and work in the EU. They are about the people who have chosen to come here because of our values, our culture, our language and our history. They are about the people who have made this country their home and have contributed to our economy and our society. The Government's plans are not just about the EU nationals; they are about the British people who are being denied the opportunity to live, work and s

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Kit Malthouse: Kit Malthouse said I must say, I'm disappointed, but not surprised, by the level of emotional rhetoric that has been displayed by the Liberal Democrats today. This is not a debate about the values of freedom, equality, justice, and human rights; it's a debate about the future of our country and the future of our nation. The future of our nation is not about the freedom of EU nationals to live and work in the UK; it's about the freedom of the British people to make their own decisions about their own country. The British people have spoken, and they have chosen to leave the EU. We must respect their decision and we must take control of our own destiny. The Liberal Democrats' opposition to this is not about the values of the EU; it's about their opposition to the will of the British people. They are trying to undermine the democratic process and to impose their own views on the country. I say to the Liberal Democrats that this is not the way to do it. We will not be swayed

...

Steve Double: Steve Double said I am deeply disappointed that the hon. Lady is still refusing to acknowledge the will of the British people. The British people voted for Brexit, and it is our duty to deliver it. We cannot continue to be held hostage by the EU's free movement policies, which have been exploited by traffickers and those seeking to exploit our country. The hon. Lady's words are nothing but a thinly veiled attempt to justify the continued exploitation of our country. We will not be swayed by emotional appeals to the heart. We will not be swayed by the rhetoric of the EU. We will take control of our borders, and we will make our own decisions about who comes into our country. That is what the British people want. We will not be lectured to by the EU about the importance of diversity and inclusivity. We will make our own decisions about who we want to welcome into our country. And we will do it on the basis of merit, not on the basis of where someone was born. The EU Settlem

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Rachael Maskell: Rachael Maskell said I am deeply disappointed by the hon. Gentleman's response. He is still stuck in the past, and he is refusing to listen to the concerns of the British people. We are not just talking about the rights of EU nationals; we are talking about the rights of the most vulnerable people in our society. We are talking about the rights of unaccompanied minors, who are being torn from their families and left to fend for themselves in our country. We are talking about the rights of those who have been trafficked and exploited in our country, and who are now being forced to live in limbo, unable to access the services they need because of the uncertainty surrounding their immigration status. The hon. Gentleman talks about the rights of the British people, but he ignores the rights of those who are most vulnerable. He talks about the need for control, but he ignores the need for compassion. We will not be swayed by the rhetoric of the EU; we will not be swayed by 

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Tim Farron: Tim Farron said I must correct the hon. Lady. I did not say that we should abandon the most vulnerable people in our society. On the contrary, I said that we should be compassionate and provide support to those who are most vulnerable. The right hon. Lady's suggestion that I am stuck in the past is quite wrong. I am a strong advocate for the rights of EU nationals, and I will continue to fight for their rights. The right hon. Lady's comments about the hon. Member for Birmingham, Ladywood (Ruth Sargent) are quite irrelevant to this debate. We are not talking about the rights of EU nationals; we are talking about the rights of the most vulnerable people in our society. We are talking about the rights of unaccompanied minors, who are being torn from their families and left to fend for themselves in our country. We are talking about the rights of those who have been trafficked and exploited in our country, and who are now being forced to live in limbo, unable to access the serv